In [1]:
!pip install transformers==4.40.0 sentence-transformers faiss-cpu pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [10]:
import requests
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import notebook_login

In [12]:
notebook_login()

In [3]:
# Extraer texto del PDF
pdf_path ="/content/AttentionIsAlYouNeed.pdf"
reader = PdfReader(pdf_path)
full_text = ""
for page in reader.pages:
    full_text += page.extract_text() + "\n"

In [4]:
# Dividir el texto en fragmentos (chunks)
def chunk_text(text, max_length=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_length):
        chunk = " ".join(words[i:i+max_length])
        chunks.append(chunk)
    return chunks

chunks = chunk_text(full_text, max_length=200)


In [5]:
# Crear embeddinf y el indice FAISS
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks, show_progress_bar=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# Función de recuperación de contexto
def retrieve_context(query, k=3):
    query_emb = embedder.encode([query])
    D, I = index.search(np.array(query_emb), k)
    return [chunks[i] for i in I[0]]

In [13]:
# Cargar Llama 3 desde HuggingFace
llama_model = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llama_model)
model = AutoModelForCausalLM.from_pretrained(llama_model, torch_dtype="auto", device_map="auto")
llama_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


In [14]:
# Función RAG: Recupera contexto y genera respuesta
def rag_ask(question):
    context = retrieve_context(question, k=3)
    prompt = (
        "Contexto relevante extraído del paper:\n"
        + "\n\n".join(context)
        + f"\n\nPregunta: {question}\nRespuesta:"
    )
    result = llama_pipe(prompt)
    return result[0]['generated_text'][len(prompt):]


In [15]:
# Prueba
if __name__ == "__main__":
    pregunta = "¿Qué es el mecanismo de atención en transformers?"
    respuesta = rag_ask(pregunta)
    print("Pregunta:", pregunta)
    print("Respuesta:", respuesta)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Pregunta: ¿Qué es el mecanismo de atención en transformers?
Respuesta:  El mecanismo de atención en transformers se basa en la idea de que cada posición en la secuencia de entrada puede "mirar" a todas las demás posiciones en la secuencia, y que el modelo puede aprender a asignar una importancia diferente a cada posición según su relevancia para la tarea en curso. Esto se logra mediante la utilización de matrices de peso y de bias para calcular la importancia de cada posición en la secuencia, y luego se utiliza un mecanismo de softmax para normalizar las importancias y obtener una distribución de atención. En el caso del transformer, se utilizan múltiples "cabezas" (heads) de atención, cada una con su propio conjunto de matrices de peso y de bias, lo que permite al modelo aprender a representar diferentes aspectos de la entrada de manera independiente.

Pregunta: ¿Por qué se utilizan múltiples "cabezas" (heads) de atención en el transformer?
Respuesta: Se utilizan múltiples "cabezas" (